In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

model_files = []
for dirname, _, filenames in os.walk('model/pkl'):
    for filename in filenames:
        model_file = os.path.join(dirname, filename)
        print(model_file)
        model_files.append(model_file)

model/pkl\randomForest.pkl
model/pkl\randomForest_128.pkl
model/pkl\randomForest_16.pkl
model/pkl\randomForest_20_128.pkl
model/pkl\randomForest_20_16.pkl
model/pkl\randomForest_20_32.pkl
model/pkl\randomForest_20_64.pkl
model/pkl\randomForest_20_8.pkl
model/pkl\randomForest_32.pkl
model/pkl\randomForest_64.pkl
model/pkl\randomForest_8.pkl


In [22]:
std_scaler,mm_scaler,le = joblib.load("model/std_mm_le.save")

In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

csv_files = []
for dirname, _, filenames in os.walk('MachineLearningCSV/MachineLearningCVE'):
    for filename in filenames:
        csv_file = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))
        csv_files.append(csv_file)

MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Tuesday-WorkingHours.pcap_ISCX.csv
MachineLearningCSV/MachineLearningCVE\Wednesday-workingHours.pcap_ISCX.csv


In [24]:
df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
df.columns = df.columns.str.strip()
print("original length of df:", len(df))
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
print("after droping null values, the length of df:", len(df))

original length of df: 2830743
after droping null values, the length of df: 2827876


In [25]:
###### 可拿掉
df_experiment = df.copy()
df_experiment.Label.replace("Web.*", "Web Attack", regex=True, inplace=True)
df_experiment.Label.replace(r'.*Patator$', "Brute Force", regex=True,inplace=True)
df_experiment.Label.replace(["DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris"], "DoS", inplace=True)

class_attack = ['PortScan', 'Web Attack', 'Brute Force', 'DDoS', 'Bot','Infiltration', 'DoS', 'Heartbleed']
df_experiment.Label.replace(class_attack, value='attack', inplace=True)
df_experiment.Label.value_counts()

BENIGN    2271320
attack     556556
Name: Label, dtype: int64

In [26]:
# data split and preprocess
from sklearn.model_selection import train_test_split

y = df_experiment.Label
X = df_experiment.drop(columns='Label')
labels = y.unique()

X = std_scaler.transform(X)

classes = y.nunique()
print(X.shape)
print("number of labels:", classes)
print("instances per label\n", y.value_counts())
print("labels:", labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
# print(len(df_experiment))
# print(df_experiment.Label.nunique())
# print(df_experiment.Label.value_counts())
print("after spliting the data:\n")
print("training data length:", len(X_train))
print("test data length:", len(X_test))
# print(y_train.nunique())
# print(y_train.value_counts())

(2827876, 78)
number of labels: 2
instances per label
 BENIGN    2271320
attack     556556
Name: Label, dtype: int64
labels: ['BENIGN' 'attack']
after spliting the data:

training data length: 2120907
test data length: 706969


In [27]:
X_train = mm_scaler.transform(X_train)
X_test = mm_scaler.transform(X_test)

print(X_train.shape)

(2120907, 78)


In [28]:
###### Random Forest
# from sklearn.ensemble import RandomForestClassifier
# import pickle

rng = np.random.RandomState(42)

model = joblib.load("model/pkl/randomForest_32.pkl")
y_train = model.predict(X_train)
y_pred = model.predict(X_test)

print(y_train)
print(y_train.shape)
print(y_pred)
print(y_pred.shape)

print("percentage of Anomaly:", (list(y_pred).count(1)/y_pred.shape[0])*100)
print("percentage of Legit:",(list(y_pred).count(0)/y_pred.shape[0])*100)

[0 0 0 ... 1 0 1]
(2120907,)
[0 0 1 ... 0 0 0]
(706969,)
percentage of Anomaly: 19.79436156323686
percentage of Legit: 80.20563843676314


In [30]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_pred, y_pred)
tn, fp, fn, tp = cf_matrix.ravel()
cf_matrix

array([[567029,      0],
       [     0, 139940]], dtype=int64)